In [1]:
import jax
from typing import Any, Callable, Sequence, Optional
from jax import lax, random, numpy as jnp
import flax
from flax import linen as nn
import optax
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sherlock.deploy import helpers

Error processing line 1 of /home/sunnykim/miniconda3/envs/sherlock/lib/python3.7/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "/home/sunnykim/miniconda3/envs/sherlock/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
2023-03-20 17:16:20.920162: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-20 17:16:20.920279: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-20 17:16:20.920291: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dl

In [2]:
class RestModel(nn.Module):
    features: Sequence[int]
  
    @nn.compact
    def __call__(self, x):
        
        x = nn.BatchNorm(use_running_average=True,
                 momentum=0.9,
                 epsilon=1e-5,
                 dtype=jnp.float32)(x)
        
        return x
    
class SubModel(nn.Module):
    features: Sequence[int]
    training: bool = True
    
    @nn.compact
    def __call__(self, x):
        
        # batchnormalisation - https://github.com/google/flax/issues/932
        x = nn.BatchNorm(use_running_average=True,
                 momentum=0.9,
                 epsilon=1e-5,
                 dtype=jnp.float32)(x)
        
        x = nn.relu(nn.Dense(self.features[0])(x))
        
        # dropout
        x = nn.Dropout(rate=0.35)(x, deterministic=True)
                
        x = nn.relu(nn.Dense(self.features[1])(x)) 
        
        # todo: add 
        
        
        return x
    
class MainModel(nn.Module):
    feature_size: int = 500
    num_classes: int = 78
        
    @nn.compact
    def __call__(self, x1, x2, x3, x4):
       
        # [1] define shape        
        y1 = SubModel([300, 300], name='char_model')(x1)      
        y2 = SubModel([200, 200], name='word_model')(x2)
        y3 = SubModel([400, 400], name='par_model')(x3)
        y4 = RestModel([27], name='rest_model')(x4)
                      
        # [2] concat submodels    
        x = jnp.concatenate((y1, y2, y3, y4), axis=-1)
        
        print("check mainmodel shape")
        print(np.shape(x))
        
        # batchnormalisation
        x = nn.BatchNorm(use_running_average=True,
                 momentum=0.9,
                 epsilon=1e-5,
                 dtype=jnp.float32)(x)
        
        # dense 1
        x = nn.relu(nn.Dense(self.feature_size)(x))
        
        # dropout
        x = nn.Dropout(rate=0.35)(x, deterministic=True)
        
        # dense 2
        x = nn.relu(nn.Dense(self.feature_size)(x))
        
        # dense w/ softmax - todo: check
        x = nn.softmax(nn.Dense(self.feature_size)(x), axis=-1)
        
        return nn.Dense(self.num_classes)(x)


In [3]:
from flax.training import train_state 

learning_rate = 0.0001
n_training_steps = 100

def create_train_state(rng):
  mainmodel = MainModel()
  params = mainmodel.init(jax.random.PRNGKey(0), jnp.ones((1, 960)), jnp.ones((1, 201)), jnp.ones((1, 400)), jnp.ones((1, 27)))
  tx=optax.adam(learning_rate=learning_rate)
  state = train_state.TrainState.create(apply_fn=mainmodel.apply,
                                        params=params,
                                        tx=tx)
  return state 



In [4]:
## RESTORE
# file - byte -> TrainState

rng = jax.random.PRNGKey(0)
rng, init_rng = jax.random.split(rng)

with open("flax_model_MAR.msgpack", mode='rb') as file:
    read_data = file.read()

new_state = create_train_state(init_rng)    
restored_state = flax.serialization.from_bytes(new_state, read_data)  ##TrainState

check mainmodel shape
(1, 927)


## Prediction

In [12]:
# todo raw data -> featurization

feature_cols = helpers.categorize_features()

X_test = pd.read_parquet('../data/data/processed/inputX.parquet')
# y_validation = pd.read_parquet('../data/data/raw/inputY_pq.parquet').values.flatten()
y_val = pd.read_parquet('../data/data/raw/inputY_pq.parquet')
y_val = y_val.values.flatten() #to ndarray


# y_val = y_val.applymap(str)
y_validation = np.array([x.lower() for x in y_val])



In [14]:
X_char = X_test[feature_cols["char"]]
X_word = X_test[feature_cols["word"]]
X_par = X_test[feature_cols["par"]]
X_rest = X_test[feature_cols["rest"]]


In [15]:
j_v_1 = jnp.array(pd.DataFrame(X_char).to_numpy())
j_v_2 = jnp.array(pd.DataFrame(X_word).to_numpy())
j_v_3 = jnp.array(pd.DataFrame(X_par).to_numpy())
j_v_4 = jnp.array(pd.DataFrame(X_rest).to_numpy())

Prediction

In [16]:
mainmodel = MainModel()
y_pred_sv = mainmodel.apply(new_state.params, j_v_1, j_v_2, j_v_3, j_v_4)

2023-03-20 17:21:13.199583: W external/org_tensorflow/tensorflow/compiler/xla/service/gpu/gemm_algorithm_picker.cc:219] Failed to find best cuBLAS algorithm, GEMM performance might be suboptimal: INTERNAL: All algorithms tried for %cublas-gemm.1 = f32[4,300]{1,0} custom-call(f32[4,960]{1,0} %Arg_0.1, f32[960,300]{1,0} %Arg_1.2), custom_call_target="__cublas$gemm", metadata={op_name="jit(dot_general)/jit(main)/dot_general[dimension_numbers=(((1,), (0,)), ((), ())) precision=None preferred_element_type=None]" source_file="/tmp/ipykernel_364921/3202350427.py" source_line=27}, backend_config="{\"alpha_real\":1,\"alpha_imag\":0,\"beta\":0,\"dot_dimension_numbers\":{\"lhs_contracting_dimensions\":[\"1\"],\"rhs_contracting_dimensions\":[\"0\"],\"lhs_batch_dimensions\":[],\"rhs_batch_dimensions\":[]},\"precision_config\":{\"operand_precision\":[\"DEFAULT\",\"DEFAULT\"]},\"epilogue\":\"DEFAULT\"}" failed. Falling back to default algorithm.  Per-algorithm errors:
  
  
  
  
  
  
  
  
  
  
  

check mainmodel shape
(4, 927)


2023-03-20 17:21:14.652855: W external/org_tensorflow/tensorflow/compiler/xla/service/gpu/gemm_algorithm_picker.cc:219] Failed to find best cuBLAS algorithm, GEMM performance might be suboptimal: INTERNAL: All algorithms tried for %cublas-gemm.1 = f32[4,500]{1,0} custom-call(f32[4,927]{1,0} %Arg_0.1, f32[927,500]{1,0} %Arg_1.2), custom_call_target="__cublas$gemm", metadata={op_name="jit(dot_general)/jit(main)/dot_general[dimension_numbers=(((1,), (0,)), ((), ())) precision=None preferred_element_type=None]" source_file="/tmp/ipykernel_364921/3202350427.py" source_line=65}, backend_config="{\"alpha_real\":1,\"alpha_imag\":0,\"beta\":0,\"dot_dimension_numbers\":{\"lhs_contracting_dimensions\":[\"1\"],\"rhs_contracting_dimensions\":[\"0\"],\"lhs_batch_dimensions\":[],\"rhs_batch_dimensions\":[]},\"precision_config\":{\"operand_precision\":[\"DEFAULT\",\"DEFAULT\"]},\"epilogue\":\"DEFAULT\"}" failed. Falling back to default algorithm.  Per-algorithm errors:
  
  
  
  
  
  
  
  
  
  
  

In [17]:
y_pred_classes = helpers._proba_to_classes(y_pred_sv, "sherlock")

y_pred_classes

array(['affiliate', 'address', 'address', 'family'], dtype=object)

Score - additional

In [18]:
print(f1_score(y_validation, y_pred_classes, average="weighted"))
print(accuracy_score(y_validation, y_pred_classes))

0.0
0.0


In [19]:
print(y_validation)
print(y_pred_classes)

['area' 'collection' 'team name' 'credit']
['affiliate' 'address' 'address' 'family']


: 